In [16]:
# from google.colab import files
# uploaded = files.upload()
#%pip install dlib

In [1]:
import cv2
import dlib

detector = dlib.get_frontal_face_detector()
new_path =os.path.join(os.getcwd(),'detected_images\\')


In [2]:
def MyRec(rgb,x,y,w,h,v=20,color=(200,0,0),thikness =2):
    """To draw stylish rectangle around the objects"""
    cv2.line(rgb, (x,y),(x+v,y), color, thikness)
    cv2.line(rgb, (x,y),(x,y+v), color, thikness)

    cv2.line(rgb, (x+w,y),(x+w-v,y), color, thikness)
    cv2.line(rgb, (x+w,y),(x+w,y+v), color, thikness)

    cv2.line(rgb, (x,y+h),(x,y+h-v), color, thikness)
    cv2.line(rgb, (x,y+h),(x+v,y+h), color, thikness)

    cv2.line(rgb, (x+w,y+h),(x+w,y+h-v), color, thikness)
    cv2.line(rgb, (x+w,y+h),(x+w-v,y+h), color, thikness)



In [3]:
def save(img,name, bbox, width=180,height=227):
    x, y, w, h = bbox
    imgCrop = img[y:h, x: w]
    imgCrop = cv2.resize(imgCrop, (width, height))
    cv2.imwrite(name+".jpg", imgCrop)




In [4]:
  # from google.colab.patches import cv2_imshow D:\project\project\images\t1.jpeg

In [5]:
def faces(imagepath):
    print(imagepath)
    frame =cv2.imread(imagepath)
    gray =cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    fit =20
    # detect the face
    for counter,face in enumerate(faces):
        count=counter
        x1, y1 = face.left(), face.top()
        x2, y2 = face.right(), face.bottom()
        cv2.rectangle(frame,(x1,y1),(x2,y2),(220,255,220),1)
        MyRec(frame, x1, y1, x2 - x1, y2 - y1, 10, (0,250,0), 3)
        # save(gray,new_path+str(counter),(x1-fit,y1-fit,x2+fit,y2+fit))
        save(gray,new_path+str(counter),(x1,y1,x2,y2))
    frame = cv2.resize(frame,(800,800))
    cv2.imshow('Group-Face Detection',frame)
    cv2.waitKey(0)
    print("done saving")
    return count+1

In [6]:
for i in os.listdir(os.path.join(os.getcwd(),'images')):
    count=faces(os.path.join(os.getcwd(),'images',i))


d:\project\project\images\group-image1.jpg
done saving
d:\project\project\images\group.jpg
done saving
d:\project\project\images\group.png
done saving
d:\project\project\images\group1.png
done saving
d:\project\project\images\image_1.jpg
done saving
d:\project\project\images\img2.jpg
done saving
d:\project\project\images\img4.jpg
done saving
d:\project\project\images\img5.jpg
done saving
d:\project\project\images\mg1.jpeg
done saving


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import IPython
import numpy as np
import os

In [8]:
config=tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(config=config)
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU')) 


True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
path = os.getcwd()

In [10]:
def stop_cuda():
    from numba import cuda 
    device = cuda.get_current_device()
    device.reset()

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
batch_size = 64
img_width = 48
img_height = 48

# print("Train : ",train_dir)
# print("Val : ",val_dir)

In [12]:
# train_generator = train_datagen.flow_from_directory(
#         train_dir,
#         target_size=(img_width,img_height),
#         batch_size=batch_size,
#         color_mode="grayscale",
#         class_mode='categorical')

# validation_generator = val_datagen.flow_from_directory(
#         val_dir,
#         target_size=(img_width,img_height),
#         batch_size=batch_size,
#         color_mode="grayscale",
#         class_mode='categorical')


In [13]:
#{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
emotions={0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

In [14]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same',input_shape=(img_height,img_width,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.3))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.3))


emotion_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))

emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.3))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))
################################################################################
emotion_model.load_weights(os.path.join(os.getcwd(),'Models/adagram_weights/emotion-reg-weight-0.8965796828269958.h5'))

In [15]:
validation_generator = val_datagen.flow_from_directory(
        os.path.join(os.getcwd(),'detected_images'),
        target_size=(img_width,img_height),
        class_mode='categorical')

Found 0 images belonging to 0 classes.


In [16]:
def create_dataset(img_folder):

    img_data_array=[]
    class_name=[]

    for file in os.listdir(os.path.join(img_folder)):

        image_path= os.path.join(img_folder, file)
        image= cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image=cv2.resize(image, (img_height, img_width))
        image=np.array(image)
        image = image.astype('float32')
        image /= 255 
        print(image.shape)
        img_data_array.append(np.reshape(image,(-1,48,48,1)))
        # class_name.append(dir1)
    return img_data_array

In [17]:
img_data_array = create_dataset(os.path.join(os.getcwd(),'detected_images'))
print(img_data_array)

(48, 48)
(48, 48)
(48, 48)
(48, 48)
(48, 48)
(48, 48)
(48, 48)
(48, 48)
(48, 48)
(48, 48)
(48, 48)
(48, 48)
(48, 48)
(48, 48)
(48, 48)
(48, 48)
(48, 48)
(48, 48)
(48, 48)
(48, 48)
[array([[[[0.6039216 ],
         [0.59607846],
         [0.59607846],
         ...,
         [0.21568628],
         [0.3372549 ],
         [0.31764707]],

        [[0.5921569 ],
         [0.5921569 ],
         [0.62352943],
         ...,
         [0.23529412],
         [0.30980393],
         [0.29411766]],

        [[0.61960787],
         [0.5921569 ],
         [0.5882353 ],
         ...,
         [0.24313726],
         [0.29411766],
         [0.2784314 ]],

        ...,

        [[0.85882354],
         [0.827451  ],
         [0.827451  ],
         ...,
         [0.8509804 ],
         [0.81960785],
         [0.76862746]],

        [[0.85882354],
         [0.827451  ],
         [0.83137256],
         ...,
         [0.8235294 ],
         [0.7764706 ],
         [0.74509805]],

        [[0.85882354],
         [0.

In [18]:
output=[]
for i in range(0,count):
    output.append(np.argmax(emotion_model.predict(img_data_array[i])))
print(output)

[3]


In [19]:
resultant_dict=dict()
for i in output:
    if i not in resultant_dict:
        resultant_dict[i]=1
    else:
        resultant_dict[i]+=1  
print(resultant_dict)


{3: 1}


In [20]:
for i in os.listdir(new_path):
    os.remove(os.path.join(os.getcwd(),'detected_images',i))

In [21]:
probability={}
for emotion,no_of_persons_emotion in resultant_dict.items():
    probability[emotions[emotion]]=(no_of_persons_emotion/count)*100
print(probability)
environment_situation=max(probability, key=probability.get)
print("The environment looks",environment_situation," : ",probability[environment_situation])

{'happy': 100.0}
The environment looks happy  :  100.0


In [22]:
# data=emotion_model.predict(img_data_array)